Liga ternária mais cara
========================================



## Introdução



Esse código encontra uma liga ternária composta por $x$A.$y$B.$z$C sendo que $x+y+z = 100\,\mathrm{g}$, $x>5\,\mathrm{g}$, $y>5\,\mathrm{g}$, $z>5\,\mathrm{g}$ e &ldquo;A&rdquo;, &ldquo;B&rdquo; e &ldquo;C&rdquo; são elementos químicos. Sendo que a junção desses compostos é chamada de "liga".


## Objetivo



Utilizar Algoritmos Genéticos para calcular a liga ternária seguindo as limitações definidas.



## Importações



In [1]:
from funcoes import populacao_lt as cria_populacao_inicial
from funcoes import individuo_lt
from funcoes import computa_preco_lt
from funcoes import funcao_objetivo_pop_lt
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_lt as funcao_cruzamento
from funcoes import mutacao_elemento_lt
from funcoes import mutacao_massa_lt
import random

## Códigos e discussão



Irei inovar, vou pensar em um indivíduo contendo cromossomos e genes. O indivíduo conterá 3 cromossomos, sendo que cada um desses conterá 2 genes. Cada cromossomo irá se referir a um elemento, sendo que, dos 2 genes presentes no cromossomo, o primeiro será destinado para o tipo do elemento e o segundo será destinado para a massa do elemento.

### Passo-a-passo

- Criar uma função que gera indivíduos viáveis, nesse caso, indivíduos em que os genes de massa somem 100 g.

- Criar uma função que gera uma população de indivíduos viáveis

- Criar uma função que calcula o fitness para o problema, o qual corresponderá ao preço final da liga com uma punição quando qualquer um dos elementos possui quantidade menor que 5.

- Criar uma função de cruzamento que mantenha a propriedade de soma 1: Se o pai e a mãe possuírem qualquer genes de massa com valores iguais, os cromossomos em que esses genes estão presentes serão trocados para formar os filhos

- Criar uma função de mutação que mantenha a propriedade de soma 1: Será divido em dois tipos de mutação, a de elemento e a de massa. Na de elemento, algum gene de elemento será trocado por um outro elemento qualquer. Na de massa, dois genes de massa serão escolhidos e trocados por valores aleatórios, tais que a soma desses valores é igual à soma dos valores anteriores.

- Utilizar a seleção por torneio de maximização.

In [2]:
# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 2000000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO_ELEMENTO = 0.2
CHANCE_MUTACAO_MASSA = 0.1

# relacionadas ao problema a ser resolvido
NUM_ELEMENTOS = 3
MASSA_MAX = 100
MASSA_MIN_ELEMENTO = 5

# preço em dólares por kilograma
preco = {
    "H": 1.39,
    "He": 24,
    "Li": 85.6,
    "Be": 857,
    "B": 3.68,
    "C": 0.122,
    "N": 0.14,
    "O": 0.154,
    "F": 2.16,
    "Ne": 240,
    "Na": 3.43,
    "Mg": 2.32,
    "Al": 1.79,
    "Si": 1.7,
    "P": 2.69,
    "S": 0.0926,
    "Cl": 0.082,
    "Ar": 0.931,
    "K": 13.6,
    "Ca": 2.35,
    "Sc": 3460,
    "Ti": 11.7,
    "V": 385,
    "Cr": 9.4,
    "Mn": 1.82,
    "Fe": 0.424,
    "Co": 32.8,
    "Ni": 13.9,
    "Cu": 6,
    "Zn": 2.55,
    "Ga": 148,
    "Ge": 1010,
    "As": 1.31,
    "Se": 21.4,
    "Br": 4.39,
    "Kr": 290,
    "Rb": 15500,
    "Sr": 6.68,
    "Y": 31,
    "Nb": 85.6,
    "Mo": 40.1,
    "Tc": 100000,
    "Ru": 10600,
    "Rh": 147000,
    "Pd": 49500,
    "Ag": 521,
    "Cd": 2.73,
    "In": 167,
    "Sn": 18.7,
    "Sb": 5.79,
    "Te": 63.5,
    "I": 35,
    "Xe": 1800,
    "Cs": 61800,
    "Ba": 0.275,
    "La": 4.92,
    "Ce": 4.71,
    "Pr": 103,
    "Nd": 57.5,
    "Pm": 460000,
    "Sm": 13.9,
    "Eu": 31.4,
    "Gd": 28.6,
    "Tb": 658,
    "Dy": 307,
    "Ho": 57.1,
    "Er": 26.4,
    "Tm": 3000,
    "Yb": 17.1,
    "Lu": 643,
    "Hf": 900,
    "Ta": 312,
    "W": 35.3,
    "Re": 4150,
    "Os": 12000,
    "Ir": 56200,
    "Pt": 27800,
    "Hg": 30.2,
    "Tl": 4200,
    "Pb": 2,
    "Bi": 6.36,
    "Po": 49200000000000,
    "Ac": 29000000000000,
    "Th": 287,
    "Pa": 280000,
    "U": 101,
    "Np": 660000,
    "Pu": 6490000,
    "Am": 750000,
    "Cm": 160000000000,
    "Bk": 185000000000,
    "Cf": 185000000000,
}

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_lt(populacao, preco, MASSA_MIN_ELEMENTO)

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(NUM_ELEMENTOS, MASSA_MAX, preco, TAMANHO_POP)

# variaveis para o hall da fama
melhor_fitness_ja_visto = 0.001
melhor_individuo_ja_visto = [['Al', 35], ['Nb', 0], ['Gd', 65]]

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação de elemento
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO_ELEMENTO:
            individuo = populacao[n]
            populacao[n] = mutacao_elemento_lt(individuo, preco)
            
    # Mutação de massa
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO_MASSA:
            individuo = populacao[n]
            populacao[n] = mutacao_massa_lt(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    preco_maior = computa_preco_lt(individuo, preco)
    if maior_fitness > melhor_fitness_ja_visto:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(melhor_individuo_ja_visto)
        print(f"Maior preço: {preco_maior}")

[['V', 39], ['Cf', 34], ['Cu', 27]]
Maior preço: 6290000015.177
[['Bk', 35], ['Tm', 57], ['Tb', 8]]
Maior preço: 6475000176.264
[['Np', 33], ['Pu', 23], ['Cf', 44]]
Maior preço: 8140171050.0
[['Cl', 64], ['Po', 17], ['Ca', 19]]
Maior preço: 836400000000.0499
[['Dy', 15], ['W', 47], ['Po', 38]]
Maior preço: 1869600000006.2642
[['Cm', 15], ['Po', 57], ['H', 28]]
Maior preço: 2806800000000.039
[['Ge', 32], ['Po', 59], ['Tl', 9]]
Maior preço: 2902800000070.1196
[['Y', 28], ['Po', 63], ['S', 9]]
Maior preço: 3099600000000.869
[['Po', 69], ['Ge', 22], ['Cl', 9]]
Maior preço: 3394800000022.221
[['Np', 19], ['Gd', 10], ['Po', 71]]
Maior preço: 3493200012540.286
[['Po', 79], ['Th', 10], ['Y', 11]]
Maior preço: 3886800000003.211
[['Po', 82], ['N', 7], ['Pt', 11]]
Maior preço: 4034400000305.801
[['Mg', 11], ['Po', 83], ['Os', 6]]
Maior preço: 4083600000072.0254
[['Po', 84], ['Tl', 7], ['Ir', 9]]
Maior preço: 4132800000535.1997
[['Be', 6], ['Tb', 7], ['Po', 87]]
Maior preço: 4280400000009.748
[['A

## Conclusão



O problema da liga ternária possui diversas condicionais e possibilidades de escolha, apesar de possuir uma resposta "óbvia", a sua resolução por algoritmos genéticos abre espaço para diversas outras abordagens em problemas diferentes. Portanto, neste notebook, foi proposta uma abordagem de algoritmos genéticos que incluísse a categoria de "cromossomo", característico por possuir dois genes, sendo que o primeiro se referia ao elemento e o segunda à massa utilizada desse elemento, a ideia dessa abordagem era poder variar essas duas informações independentemente sem perder a relação entre elas. Portanto, a partir disso, foi imposta a restrição de massa máxima nas funções de criação da população, cruzamento e mutação. A condição de massa mínima dos elementos foi imposta como punição na função objetivo e, consequentemente, nos métodos de seleção. Por fim, foram obtidos resultados razoáveis para o problema, sendo que, em uma das simulações, o resultado obtido foi de uma liga ternária contendo $11 g$ de $Ac$, $6 g$ de $Bi$ e $83 g$ de $Po$, totalizando um preço de $4402600000000,038$ dinheiros.

## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.



In [5]:
from funcoes import individuo_lt
from funcoes import populacao_lt
from funcoes import computa_preco_lt
from funcoes import funcao_objetivo_lt
from funcoes import cruzamento_lt
from funcoes import mutacao_elemento_lt
from funcoes import mutacao_massa_lt

In [6]:
a = individuo_lt(3, 100, preco)
b = individuo_lt(3, 100, preco)
print(a)
print(b)

[['Ac', 12], ['N', 30], ['Ni', 58]]
[['Ce', 54], ['Er', 6], ['Os', 40]]


In [7]:
cruzamento_lt(a, b)

([['Ac', 12], ['N', 30], ['Ni', 58]], [['Ce', 54], ['Er', 6], ['Os', 40]])

In [8]:
computa_preco_lt(a, preco)

348000000000.8104

In [9]:
funcao_objetivo_lt(a, preco)

TypeError: funcao_objetivo_lt() missing 1 required positional argument: 'massa_min_elemento'

In [ ]:
populacao_lt(3, 100, preco, 20)

In [ ]:
mutacao_elemento_lt(a, preco)

In [ ]:
mutacao_massa_lt(a)

In [ ]:
l = [1, 2, 3, 4]